In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 🎯 vibert GRI Classification with Stratified Split & Focal Loss

In [ ]:
!pip install -q transformers datasets scikit-learn

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("Input of labelled file")
df = df.dropna(subset=["Sentences", "GRI"])
df["GRI"] = df["GRI"].astype(str).str.replace(" ", "")
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["GRI"])

In [ ]:
from sklearn.model_selection import train_test_split

def stratified_split_data(df, label_column="label", test_size=0.1, val_size=0.1, random_state=42):
    train_val_df, test_df = train_test_split(df, test_size=test_size, stratify=df[label_column], random_state=random_state)
    train_df, val_df = train_test_split(train_val_df, test_size=val_size / (1 - test_size), stratify=train_val_df[label_column], random_state=random_state)
    return train_df.reset_index(drop=True), val_df.reset_index(drop=True), test_df.reset_index(drop=True)

train_df, val_df, test_df = stratified_split_data(df)

In [ ]:
# from sklearn.utils.class_weight import compute_class_weight
# import numpy as np
# import torch

# class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_df['label']), y=train_df['label'])
# class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

In [ ]:
from transformers import AutoTokenizer
from torch.utils.data import Dataset
import torch

tokenizer = AutoTokenizer.from_pretrained("trituenhantaoio/bert-base-vietnamese-uncased", use_fast=False)

from torch.utils.data import Dataset

class TorchDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenizer(train_df['Sentecnes'].tolist(), truncation=True, padding=True, max_length=256, return_tensors="pt")
val_encodings = tokenizer(val_df['Sentences'].tolist(), truncation=True, padding=True, max_length=256, return_tensors="pt")

train_dataset = TorchDataset(train_encodings, torch.tensor(train_df['label'].values))
val_dataset = TorchDataset(val_encodings, torch.tensor(val_df['label'].values))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("trituenhantaoio/bert-base-vietnamese-uncased", num_labels=df['label'].nunique())

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at trituenhantaoio/bert-base-vietnamese-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

In [ ]:
import torch
import torch.nn.functional as F
from transformers import Trainer

class FocalLossTrainer(Trainer):
    def __init__(self, *args, alpha=None, gamma=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha = alpha
        self.gamma = gamma

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Move alpha to the same device as logits
        alpha_on_device = self.alpha.to(logits.device)

        ce_loss = F.cross_entropy(logits, labels, weight=alpha_on_device)
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        return (focal_loss, outputs) if return_outputs else focal_loss

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_df["label"]),
    y=train_df["label"]
)
alpha_tensor = torch.tensor(class_weights, dtype=torch.float)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_micro": f1_score(labels, preds, average="micro"),
        "f1_macro": f1_score(labels, preds, average="macro"),
        "precision_micro": precision_score(labels, preds, average="micro", zero_division=0),
        "recall_micro": recall_score(labels, preds, average="micro", zero_division=0),
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="output_folder",
    do_train=True,
    do_eval=True,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=15,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1_macro",
    greater_is_better=True,
    logging_dir="./train_logs_gri",
    logging_steps=50
)

trainer = FocalLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)],
    alpha=alpha_tensor,  # Add alpha here
    gamma=2.4             # Focal loss focusing parameter
)

trainer.train()

/tmp/ipython-input-9-3087256054.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FocalLossTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lamtruongnguyen2030 (leighton2030) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,Precision Micro,Recall Micro
1,0.260400,0.121539,0.794038,0.794038,0.793824,0.794038,0.794038
2,0.091600,0.084817,0.791328,0.791328,0.791633,0.791328,0.791328
3,0.063200,0.094311,0.796748,0.796748,0.797612,0.796748,0.796748
4,0.029100,0.110667,0.785908,0.785908,0.789360,0.785908,0.785908
5,0.017600,0.123325,0.785908,0.785908,0.785297,0.785908,0.785908
6,0.014200,0.116717,0.788618,0.788618,0.788102,0.788618,0.788618
7,0.008600,0.146952,0.775068,0.775068,0.776601,0.775068,0.775068


TrainOutput(global_step=1295, training_loss=0.1321852043558732, metrics={'train_runtime': 1014.1199, 'train_samples_per_second': 43.663, 'train_steps_per_second': 2.736, 'total_flos': 2718780727799808.0, 'train_loss': 0.1321852043558732, 'epoch': 7.0})

In [ ]:
# prompt: evaluate on each class

import numpy as np
from sklearn.metrics import classification_report

test_encodings = tokenizer(test_df['Câu'].tolist(), truncation=True, padding=True, max_length=256, return_tensors="pt")
test_dataset = TorchDataset(test_encodings, torch.tensor(test_df['label'].values))

# Make predictions on the test set
predictions = trainer.predict(test_dataset)

# Get predicted labels
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Get true labels
true_labels = predictions.label_ids

# Get the mapping from label index back to original GRI label
label_mapping = {idx: label for label, idx in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}

# Sort the target_names based on the integer labels
target_names = [label_mapping[i] for i in sorted(label_mapping.keys())]

# Evaluate on each class
report = classification_report(true_labels, predicted_labels, target_names=target_names, zero_division=0)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

# 4. Detailed report for each class
print("\n📊 BClassification report on test set:")
print (report)
# 5. Precision, Recall, F1 (macro)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='macro')
print("\n📌 Macro evaluation results:")
print(f"🔹 Precision (macro): {precision:.4f}")
print(f"🔹 Recall (macro):    {recall:.4f}")
print(f"🔹 F1-score (macro):  {f1:.4f}")


📊 Báo cáo phân loại trên tập test:
              precision    recall  f1-score   support

      GRI401       0.79      0.79      0.79        24
    GRI401-2       1.00      0.80      0.89        25
      GRI402       0.65      0.60      0.62        25
      GRI403       0.61      0.80      0.69        25
      GRI404       0.74      0.68      0.71        25
    GRI404-2       0.83      0.62      0.71        24
    GRI404-3       0.50      0.64      0.56        25
      GRI406       1.00      1.00      1.00        25
      GRI407       0.74      0.83      0.78        24
      GRI408       1.00      1.00      1.00        24
    GRI412-2       0.64      0.56      0.60        25
    GRI413-1       0.94      0.71      0.81        24
      GRI417       1.00      1.00      1.00        25
      GRI418       0.87      0.83      0.85        24
      GRI419       0.72      0.92      0.81        25

    accuracy                           0.79       369
   macro avg       0.80      0.79      0.79 

# 🎯 XLM-R GRI Classification with Stratified Split & Weighted Loss 

In [ ]:
!pip install -q transformers datasets scikit-learn

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("input_of_labeled_file")
df = df.dropna(subset=["Sentences", "GRI"])
df["GRI"] = df["GRI"].astype(str).str.replace(" ", "")
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["GRI"])

In [ ]:
from sklearn.model_selection import train_test_split

def stratified_split_data(df, label_column="label", test_size=0.1, val_size=0.1, random_state=42):
    train_val_df, test_df = train_test_split(df, test_size=test_size, stratify=df[label_column], random_state=random_state)
    train_df, val_df = train_test_split(train_val_df, test_size=val_size / (1 - test_size), stratify=train_val_df[label_column], random_state=random_state)
    return train_df.reset_index(drop=True), val_df.reset_index(drop=True), test_df.reset_index(drop=True)

train_df, val_df, test_df = stratified_split_data(df)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_df['label']), y=train_df['label'])
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

In [ ]:
from transformers import AutoTokenizer
from torch.utils.data import Dataset

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base", use_fast=False)

from torch.utils.data import Dataset

class TorchDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenizer(train_df['Sentecnes'].tolist(), truncation=True, padding=True, max_length=256, return_tensors="pt")
val_encodings = tokenizer(val_df['Sentences'].tolist(), truncation=True, padding=True, max_length=256, return_tensors="pt")

train_dataset = TorchDataset(train_encodings, torch.tensor(train_df['label'].values))
val_dataset = TorchDataset(val_encodings, torch.tensor(val_df['label'].values))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=df['label'].nunique())

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

In [ ]:
from transformers import Trainer
from torch.nn import CrossEntropyLoss

class WeightedLossTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # ✅ THÊM **kwargs
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Gắn class weights vào CrossEntropyLoss
        loss_fn = CrossEntropyLoss(weight=self.class_weights.to(logits.device))
        loss = loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_micro": f1_score(labels, preds, average="micro"),
        "f1_macro": f1_score(labels, preds, average="macro"),
        "precision_micro": precision_score(labels, preds, average="micro", zero_division=0),
        "recall_micro": recall_score(labels, preds, average="micro", zero_division=0),
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="output_folder",
    do_train=True,
    do_eval=True,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1_macro",
    greater_is_better=True,
    logging_dir="./train_logs_gri",
    logging_steps=50
)

trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    class_weights=class_weights_tensor,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,Precision Micro,Recall Micro
1,1.597500,0.930786,0.720867,0.720867,0.703517,0.720867,0.720867
2,0.747600,0.607209,0.775068,0.775068,0.778322,0.775068,0.775068
3,0.647000,0.558665,0.794038,0.794038,0.796649,0.794038,0.794038
4,0.494100,0.545806,0.794038,0.794038,0.796750,0.794038,0.794038
5,0.403800,0.533699,0.807588,0.807588,0.805609,0.807588,0.807588
6,0.370200,0.561089,0.794038,0.794038,0.796632,0.794038,0.794038
7,0.356100,0.599340,0.780488,0.780488,0.779363,0.780488,0.780488
8,0.288400,0.616214,0.783198,0.783198,0.783680,0.783198,0.783198


TrainOutput(global_step=1480, training_loss=0.6543493093671026, metrics={'train_runtime': 1415.2736, 'train_samples_per_second': 20.858, 'train_steps_per_second': 1.307, 'total_flos': 3107177974628352.0, 'train_loss': 0.6543493093671026, 'epoch': 8.0})

In [ ]:
# prompt: evaluate on each class

import numpy as np
from sklearn.metrics import classification_report

test_encodings = tokenizer(test_df['Sentences'].tolist(), truncation=True, padding=True, max_length=256, return_tensors="pt")
test_dataset = TorchDataset(test_encodings, torch.tensor(test_df['label'].values))

# Make predictions on the test set
predictions = trainer.predict(test_dataset)

# Get predicted labels
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Get true labels
true_labels = predictions.label_ids

# Get the mapping from label index back to original GRI label
label_mapping = {idx: label for label, idx in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}

# Sort the target_names based on the integer labels
target_names = [label_mapping[i] for i in sorted(label_mapping.keys())]

# Evaluate on each class
report = classification_report(true_labels, predicted_labels, target_names=target_names, zero_division=0)
report

'              precision    recall  f1-score   support\n\n      GRI401       0.85      0.71      0.77        24\n    GRI401-2       0.91      0.84      0.88        25\n      GRI402       0.49      0.72      0.58        25\n      GRI403       0.80      0.64      0.71        25\n      GRI404       0.71      0.80      0.75        25\n    GRI404-2       0.81      0.71      0.76        24\n    GRI404-3       0.57      0.64      0.60        25\n      GRI406       0.96      1.00      0.98        25\n      GRI407       0.83      0.83      0.83        24\n      GRI408       1.00      1.00      1.00        24\n    GRI412-2       0.67      0.56      0.61        25\n    GRI413-1       0.79      0.79      0.79        24\n      GRI417       1.00      1.00      1.00        25\n      GRI418       0.84      0.88      0.86        24\n      GRI419       0.83      0.76      0.79        25\n\n    accuracy                           0.79       369\n   macro avg       0.80      0.79      0.79       369\nweigh

In [ ]:
# prompt: evaluate on each class

from sklearn.metrics import precision_recall_fscore_support

# 4. Detailed report for each class
print("\n📊 Classification report on test set:")
print (report)
# 5. Precision, Recall, F1 (macro)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='macro')
print("\n📌 KMacro evaluation results:")
print(f"🔹 Precision (macro): {precision:.4f}")
print(f"🔹 Recall (macro):    {recall:.4f}")
print(f"🔹 F1-score (macro):  {f1:.4f}")


📊 Báo cáo phân loại trên tập test:
              precision    recall  f1-score   support

      GRI401       0.85      0.71      0.77        24
    GRI401-2       0.91      0.84      0.88        25
      GRI402       0.49      0.72      0.58        25
      GRI403       0.80      0.64      0.71        25
      GRI404       0.71      0.80      0.75        25
    GRI404-2       0.81      0.71      0.76        24
    GRI404-3       0.57      0.64      0.60        25
      GRI406       0.96      1.00      0.98        25
      GRI407       0.83      0.83      0.83        24
      GRI408       1.00      1.00      1.00        24
    GRI412-2       0.67      0.56      0.61        25
    GRI413-1       0.79      0.79      0.79        24
      GRI417       1.00      1.00      1.00        25
      GRI418       0.84      0.88      0.86        24
      GRI419       0.83      0.76      0.79        25

    accuracy                           0.79       369
   macro avg       0.80      0.79      0.79 

# 🎯PHOBERT GRI Classification with Stratified Split & Weighted Loss 

In [ ]:
!pip install -q transformers datasets scikit-learn

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("Input of labelled file")
df = df.dropna(subset=["Sentences", "GRI"])
df["GRI"] = df["GRI"].astype(str).str.replace(" ", "")
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["GRI"])

In [ ]:
from sklearn.model_selection import train_test_split

def stratified_split_data(df, label_column="label", test_size=0.1, val_size=0.1, random_state=42):
    train_val_df, test_df = train_test_split(df, test_size=test_size, stratify=df[label_column], random_state=random_state)
    train_df, val_df = train_test_split(train_val_df, test_size=val_size / (1 - test_size), stratify=train_val_df[label_column], random_state=random_state)
    return train_df.reset_index(drop=True), val_df.reset_index(drop=True), test_df.reset_index(drop=True)

train_df, val_df, test_df = stratified_split_data(df)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_df['label']), y=train_df['label'])
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

In [ ]:
from transformers import AutoTokenizer
from torch.utils.data import Dataset

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

from torch.utils.data import Dataset

class TorchDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenizer(train_df['Senteneces'].tolist(), truncation=True, padding=True, max_length=256, return_tensors="pt")
val_encodings = tokenizer(val_df['Sentences'].tolist(), truncation=True, padding=True, max_length=256, return_tensors="pt")

train_dataset = TorchDataset(train_encodings, torch.tensor(train_df['label'].values))
val_dataset = TorchDataset(val_encodings, torch.tensor(val_df['label'].values))

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=df['label'].nunique())

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

In [ ]:
from transformers import Trainer
from torch.nn import CrossEntropyLoss

class WeightedLossTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # ✅ THÊM **kwargs
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Gắn class weights vào CrossEntropyLoss
        loss_fn = CrossEntropyLoss(weight=self.class_weights.to(logits.device))
        loss = loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_micro": f1_score(labels, preds, average="micro"),
        "f1_macro": f1_score(labels, preds, average="macro"),
        "precision_micro": precision_score(labels, preds, average="micro", zero_division=0),
        "recall_micro": recall_score(labels, preds, average="micro", zero_division=0),
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="output_folder",
    do_train=True,
    do_eval=True,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1_macro",
    greater_is_better=True,
    logging_dir="./train_logs_gri",
    logging_steps=50
)

trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    class_weights=class_weights_tensor,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,Precision Micro,Recall Micro
1,1.536300,1.097953,0.731707,0.731707,0.717958,0.731707,0.731707
2,0.780500,0.668480,0.788618,0.788618,0.789583,0.788618,0.788618
3,0.627800,0.573373,0.813008,0.813008,0.810585,0.813008,0.813008


In [ ]:
# prompt: evaluate on each class

import numpy as np
from sklearn.metrics import classification_report

test_encodings = tokenizer(test_df['Sentences'].tolist(), truncation=True, padding=True, max_length=256, return_tensors="pt")
test_dataset = TorchDataset(test_encodings, torch.tensor(test_df['label'].values))

# Make predictions on the test set
predictions = trainer.predict(test_dataset)

# Get predicted labels
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Get true labels
true_labels = predictions.label_ids

# Get the mapping from label index back to original GRI label
label_mapping = {idx: label for label, idx in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}

# Sort the target_names based on the integer labels
target_names = [label_mapping[i] for i in sorted(label_mapping.keys())]

# Evaluate on each class
report = classification_report(true_labels, predicted_labels, target_names=target_names, zero_division=0)
report

In [ ]:
print(f"✅ Accuracy on the test set: {test_acc:.4f}")

# 4. Detailed report for each class
print("\n📊 Classification report on test set:")
print (report)
# 5. Precision, Recall, F1 (macro)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='macro')
print("\n📌 Macro evaluation results:")
print(f"🔹 Precision (macro): {precision:.4f}")
print(f"🔹 Recall (macro):    {recall:.4f}")
print(f"🔹 F1-score (macro):  {f1:.4f}")